In [5]:
!git clone https://github.com/routsourav1729/CUB_200_2011.git


Cloning into 'CUB_200_2011'...
remote: Enumerating objects: 12195, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 12195 (delta 0), reused 4 (delta 0), pack-reused 12191
Receiving objects: 100% (12195/12195), 1.05 GiB | 44.54 MiB/s, done.
Updating files: 100% (11790/11790), done.


In [6]:
!pip install efficientnet-pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=e18e5d5b17f6d93b58d9c8f76e666ba328ca7ddad7bf68d5b311b39d2e44ff52
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch
ERROR: Operation cancelled by user


In [ ]:
pip install "torchvision>0.16.0"

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torch import optim

from sklearn.metrics import accuracy_score

In [2]:
# Image transformations
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=299, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=299),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'test':
    transforms.Compose([
        transforms.Resize(size=299),
        transforms.CenterCrop(size=299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
# Datasets from folders
batch_size = 32
data = {
    'train':
    datasets.ImageFolder(root='/content/CUB_200_2011/train', transform=image_transforms['train']),
    'test':
    datasets.ImageFolder(root='/content/CUB_200_2011/test', transform=image_transforms['test']),
}

# Dataloader iterators, make sure to shuffle
dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'test': DataLoader(data['test'], batch_size=batch_size, shuffle=True)
}

In [4]:
model = models.efficientnet_b2( weights='IMAGENET1K_V1')
# Freeze model weights
for param in model.parameters():
    param.requires_grad = False

print(model.parameters())

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 142MB/s]


<generator object Module.parameters at 0x7f28c1f41690>


In [13]:
print(model)


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [5]:
# Add on classifier
model.classifier[1] = nn.Linear(1408, 200)

total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

7,982,794 total parameters.
281,800 training parameters.


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training the model
num_epochs = 20
training_loss_list = []
validation_accuracy_list = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        # print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataloaders['train'])
    training_loss_list.append(average_loss)

    # Validation after each epoch
    model.eval()
    with torch.no_grad():
        all_preds = []
        all_labels = []
        for inputs, labels in dataloaders['test']:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    validation_accuracy_list.append(accuracy)

    print(f"Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {average_loss:.4f}, Test Accuracy: {accuracy:.4f}")


cuda


KeyboardInterrupt: 

In [7]:


# Initialize model and move to device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training the model
num_epochs = 20
training_loss_list = []
validation_accuracy_list = []
training_accuracy_list = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    average_loss = total_loss / len(dataloaders['train'])
    training_loss_list.append(average_loss)
    training_accuracy = 100 * correct_train / total_train
    training_accuracy_list.append(training_accuracy)

    # Validation after each epoch
    model.eval()
    with torch.no_grad():
        all_preds = []
        all_labels = []
        for inputs, labels in dataloaders['test']:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    validation_accuracy_list.append(accuracy)

    print(f"Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {average_loss:.4f}, Train Accuracy: {training_accuracy:.2f}%, Test Accuracy: {accuracy:.4f}")


Epoch 1/20, Average Training Loss: 3.8872, Train Accuracy: 28.95%, Test Accuracy: 0.4941
Epoch 2/20, Average Training Loss: 2.3950, Train Accuracy: 58.43%, Test Accuracy: 0.5777
Epoch 3/20, Average Training Loss: 1.7718, Train Accuracy: 68.47%, Test Accuracy: 0.6101
Epoch 4/20, Average Training Loss: 1.4351, Train Accuracy: 73.92%, Test Accuracy: 0.6179
Epoch 5/20, Average Training Loss: 1.2202, Train Accuracy: 77.06%, Test Accuracy: 0.6284
Epoch 6/20, Average Training Loss: 1.0799, Train Accuracy: 79.41%, Test Accuracy: 0.6448
Epoch 7/20, Average Training Loss: 0.9471, Train Accuracy: 82.27%, Test Accuracy: 0.6519
Epoch 8/20, Average Training Loss: 0.8503, Train Accuracy: 83.67%, Test Accuracy: 0.6438


KeyboardInterrupt: 